# TPS_OCT_21_EDA

1.  loading train data and test data
1.  data description
1.  reduce memory usage
1.  visualization
1.  feature importances
1.  summary
1.  conclusion


In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
def check(df):
    col_list = train.columns.values
    rows = []
    for col in col_list:
        tmp = (col,
              train[col].dtype,
              train[col].isnull().sum(),
              train[col].count(),
              train[col].nunique(),
              train[col].unique())
        rows.append(tmp)
    df = pd.DataFrame(rows) 
    df.columns = ['feature','dtype','nan','count','nunique','unique']
    return df

check(train)

In [ ]:
def check(df):
    col_list = test.columns.values
    rows = []
    for col in col_list:
        tmp = (col,
              test[col].dtype,
              test[col].isnull().sum(),
              test[col].count(),
              test[col].nunique(),
              test[col].unique())
        rows.append(tmp)
    df = pd.DataFrame(rows) 
    df.columns = ['feature','dtype','nan','count','nunique','unique']
    return df

check(test)

In [ ]:
train.info(), test.info()

In [ ]:
# features

feature = [col for col in train.columns if col not in ['id','target']]

cat_int = []
cat_float= []

for col in train[feature]:
    if 'int' in str(train[col]):
        cat_int.append(col)
    else:
        cat_float.append(col)
        
print('integer features : ', cat_int)
print()
print('float features : ', cat_float)

---------

# How to reduce memory usage 💡

In [ ]:
# time counter ⏳
import time
start_1 = time.perf_counter()

In [ ]:
# Reduce memory usage

# train features
for col in train.columns:
    if train[col].dtype == "float64":
        train[col]=pd.to_numeric(train[col], downcast="float")
    if train[col].dtype == "int64":
        train[col]=pd.to_numeric(train[col], downcast="integer")
    

# test features
for col in test.columns:
    if test[col].dtype == "float64":
        test[col]=pd.to_numeric(test[col], downcast="float")
    if train[col].dtype == "int64":
        test[col]=pd.to_numeric(test[col], downcast="integer")  
        
train.info(), test.info()

In [ ]:
# bool change int8

col_1 = ['f' + str(i) for i in range(242, 285)] + ['f22','f43'] + ['target']
train[col_1] = train[col_1].astype('int8')

col_2 = ['f' + str(i) for i in range(242, 285)] + ['f22','f43'] 
test[col_2] = test[col_2].astype('int8')

train.info(), test.info()

In [ ]:
# done ⌛
time_end = time.perf_counter()
process_time = time_end - start_1
print("process_time:{0}".format(process_time) )

-----------

# VISUALIZATION 📊

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_theme(style = 'darkgrid')

In [ ]:
# target
sns.countplot(x = train['target'], color='#DC143C')
plt.legend()

In [ ]:
i = 22
sns.countplot(x=train['f'+str(i)], color='#663399',  label='train')
sns.countplot(x=test['f'+str(i)], color='#FFD700',  label='test')
plt.legend()

In [ ]:
i = 43
sns.countplot(x=train['f'+str(i)], color='#663399',  label='train')
sns.countplot(x=test['f'+str(i)], color='#FFD700',  label='test')
plt.legend()

-------------

# feature importances 👀

* It could take a while
* approximately one hour

In [ ]:
train = train.drop('id' ,axis = 1)

In [ ]:
X = train.drop('target', axis = 1).copy()
y = train['target'].copy()

del train
del test

In [ ]:
# time counter ⏳
start = time.perf_counter()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
clf.fit(X,y)

In [ ]:
labels = X.columns.values
importances = clf.feature_importances_
idx = np.argsort(importances)
print(idx)

In [ ]:
# done ⌛
time_end = time.perf_counter()
process_time = time_end - start
print("process_time:{0}".format(process_time) )

-----------

In [ ]:
imp = pd.DataFrame()
imp['importance'] = importances
imp = imp['importance']
imp.head()

In [ ]:
plt.figure(figsize = (50,50))
plt.title('feature importances')
plt.barh(range(imp.shape[0]), imp[idx])
plt.show()

Oh! It's very small.🔎

In [ ]:
plt.figure(figsize = (20,10))
plt.title('feature importances')
imp.nlargest(25).plot(kind='barh')
plt.show()

much better 👍

--------------

# Summary ✍
1. dataset size : train 1000K rows and 287 columns , test 500k rows and 286 columns
1. train values : no missing values, numeric only.
1. test values : no missing values, numeric only.
1. values dtype : float64 , int64 (f22, f43 f242...f284), int32(id)
1. target values : binary (0 or 1).
1. importances features : the most important feature is f22.


# Conclusion 
1. It is large data.
1. Target's positive value is almost the same as negative values.
1. It will be needed reduce memory usage.
1. It might take a while to figure out.